<a href="https://colab.research.google.com/github/estebanhernandezr/DNA-compression/blob/main/Copia_de_DEFLATEalgorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install bitarray

In [ ]:
from bitarray import *
from heapq import heappush, heappop

In [ ]:
class heapNode(object):
    def __init__(self):
        self.left = None
        self.right = None
        self.counts = 0
        self.twelveBitInteger = ""
    def __lt__(self, other):
        return self.counts < other.counts

In [ ]:
n = 50
Ls = 15
pad_symb = 37

def data_from_file(filename: str):
    with open(filename, 'rb') as input_file:
        filedata = input_file.read()
    return filedata

def file_from_bin(filename: str, newBuffer):
    with open(filename, 'wb') as outFile:
        outFile.write(newBuffer)
        return None

def bin_from_file(filename: str):
    filedata = bitarray(endian='big')
    with open(filename, 'rb') as input_file:
        filedata.fromfile(input_file)
    return filedata


def iniPad(symb, padsize, cad):
    pad = bytearray(chr(symb), 'utf-8')
    for i in range(padsize-1):
        pad.append(pad[0])
    return pad + cad

def reproducible_extension(search, lookahead):
    pos = -1
    size = 0
    char = ''
    for prefixsize in range(1, min(n-Ls, len(lookahead))):
        prefix = lookahead[:prefixsize]
        p = search.rfind(prefix, 0, (n-Ls)+prefixsize-1)
        if p >= 0:
            pos = p
            size = prefixsize
            char = lookahead[size]
        else:
            break
    return pos, size, char

def get_distances(cad):
    pcad = iniPad(cad[0], n-Ls, cad)
    distances = []
    j: int = 0
    while j < len(pcad)-(n-Ls):
        #print(pcad[j:j+n], "|", pcad[j:j+n-Ls], " --> ", pcad[j+n-Ls:j+n])
        pos, size, char = reproducible_extension(pcad[j:j+n], pcad[j+n-Ls:j+n])
        #print(pos, size)
        if pos >= 0:
          distances.append(pos)
        j += max(1,size)#+1)
    return distances

def bl_count_from_distances(distances):
    counts = {}
    for distance in distances:
        if distance in counts:
            counts[distance]+=1
        else:
            counts[distance] = 1
    return counts

def heap_from_dictionary(counts):
    heap = []
    for key in counts:
        node = heapNode()
        node.twelveBitInteger = key
        node.counts = counts[key]
        heappush(heap, node)
    #for node in heap:
    #    print(node.twelveBitInteger, ":", node.counts)
    return heap

def create_huffman_tree(counts):
    heap = heap_from_dictionary(counts)
    root = None
    while len(heap) > 1:
        smallestNode = heappop(heap)
        secondSmallestNode = heappop(heap)

        newRoot = heapNode()
        newRoot.counts = smallestNode.counts + secondSmallestNode.counts
        newRoot.twelveBitInteger = ''
        newRoot.left = smallestNode
        newRoot.right = secondSmallestNode
        root = newRoot
        heappush(heap, newRoot)
    return root

codeDictionary = {}
def dictionary_from_tree(root, path, codes):
    if root.left is None and root.right is None:
        #print(root.twelveBitInteger, ": ", root.counts)
        codes['{0:012b}'.format(root.twelveBitInteger)] = path
        return
    else:
        if root.left is not None:
            dictionary_from_tree(root.left, path+"0", codes)
        if root.right is not None:
            dictionary_from_tree(root.right, path+"1", codes)

treeBuffer = bitarray(endian='big')
def codify_huffman_tree(root, buffer):
    if root.left is None and root.right is None:
        buffer.append(False) # If current node is leaf then False (internal)
        bin_distance = '{0:012b}'.format(root.twelveBitInteger)
        for bit in bin_distance:
            if bit == '1':
                buffer.append(True)
            else:
                buffer.append(False)
    else:
        buffer.append(True) # If current node is internal then True
        codify_huffman_tree(root.left, buffer)
        codify_huffman_tree(root.right, buffer)

def decodify_huffman_tree(buffer):
    nextBit = buffer.pop(0)
    if nextBit == False: # if leaf node
        root = heapNode()
        binaryString = ''
        for i in range(12):# read next twelve bytes to get full code
            bit = buffer.pop(0)
            if bit == True:
                binaryString+="1"
            else:
                binaryString+="0"
        root.twelveBitInteger = int(binaryString, 2)
    else:
        root = heapNode()
        root.left = decodify_huffman_tree(buffer)
        root.right = decodify_huffman_tree(buffer)
    return root

outputBuffer = bitarray(endian='big')
def codify_cad(cad, root, buffer):
    pcad = iniPad(cad[0], n-Ls, cad)
    i: int = 0
    while i < len(pcad)-(n-Ls):
        pos, size, char = reproducible_extension(pcad[i:i+n], pcad[i+n-Ls:i+n])
        if pos >= 0 and size > 1:
            outputBuffer.append(True)
            if '{0:012b}'.format(pos) in codeDictionary:
                huffman_code = codeDictionary['{0:012b}'.format(pos)]
                #print("huffman_code:", huffman_code, " size:", '{0:04b}'.format(size))
                bin_code = huffman_code + '{0:04b}'.format(size)
                #print("bin_code:= ", bin_code)
                #print(type(bin_code))
                for bit in bin_code:
                    if bit == '1':
                        outputBuffer.append(True)
                    else:
                        outputBuffer.append(False)
                i += size#+1
        else:
            outputBuffer.append(False)
            outputBuffer.frombytes(bytes([pcad[i+n-Ls]])) # Literal symbol
            i += 1

def compress(filename: str):
    filedata = data_from_file(filename) #CHECKED
    distances = get_distances(filedata) #CHECKED
    #print(distances)
    counts = bl_count_from_distances(distances) #CHECKED
    #print(counts)
    huffman_tree = create_huffman_tree(counts) #CHECKED

    dictionary_from_tree(huffman_tree, '', codeDictionary) #CHECKED
    #print(codeDictionary)
    codify_huffman_tree(huffman_tree, treeBuffer) #CHECKED
    #print(treeBuffer)
    codify_cad(filedata, huffman_tree, outputBuffer) #CHECKED
    #print(outputBuffer)

    newBuffer = treeBuffer + outputBuffer
    newBuffer.fill()
    #print(newBuffer)

    file_from_bin('compressed_file', newBuffer)
    print("File compressed succesfully")
    return filedata

def decompress(filename, outputFile):
    filedata = bin_from_file(filename)
    codeDictionary = {}
    root = decodify_huffman_tree(filedata)
    dictionary_from_tree(root, '', codeDictionary)

    outputBuffer = iniPad(37, n-Ls, bytes())
    k: int = 0
    while len(filedata) >= 9:
        huffman_pair = filedata.pop(0)
        if not huffman_pair:
            byte = filedata[0:8].tobytes()
            outputBuffer += byte
            del filedata[0:8]
            k += 1
        else:
            curbitsubstring = ''
            stop = False
            while stop == False: # compare each bit subsequence and see if it is in the huffman table
                bit = filedata.pop(0)
                if bit == True:
                    curbitsubstring+="1"
                else:
                    curbitsubstring+="0"
                for key in codeDictionary:
                    if codeDictionary[key] == str(curbitsubstring):
                        twelvebitnumber = key
                        stop = True

            bestLengthBinary = ''
            for i in range(0,4):
                bit = filedata.pop(0)
                if bit == True:
                    bestLengthBinary+='1'
                else:
                    bestLengthBinary+='0'

            bestDistance = int(twelvebitnumber, 2)
            bestLength = int(bestLengthBinary, 2)
            for i in range(bestLength):
                outputBuffer.append(outputBuffer[k+bestDistance+i])
            k += bestLength

    file_from_bin(outputFile, outputBuffer[n-Ls:])
    print("File decompressed succesfully")
    return outputBuffer[n-Ls:]

In [ ]:
filename: str = 'Probabilidad.pdf'
data = compress(filename)

out_filename: str = 'uncompressed_file'
decompressed = decompress('compressed_file', out_filename)

print(data == decompressed)
print(data)
print(decompressed)

File compressed succesfully
File decompressed succesfully
True
b'%PDF-1.5\n%\xbf\xf7\xa2\xfe\n15 0 obj\n<< /Linearized 1 /L 182178 /H [ 1069 203 ] /O 19 /E 123527 /N 5 /T 181820 >>\nendobj\n                                                                                                           \n16 0 obj\n<< /Type /XRef /Length 89 /Filter /FlateDecode /DecodeParms << /Columns 5 /Predictor 12 >> /W [ 1 3 1 ] /Index [ 15 69 ] /Info 13 0 R /Root 17 0 R /Size 84 /Prev 181821                /ID [<04831c57eb77db4b54c4e9341128264c><ce8e7fe40d0bb2e0298d55979222034c>] >>\nstream\nx\x9ccbd`\xe0g`b``8\t"\x99\x95@$\xa31X\xe44\x98\x9d\x0f"\xc5\xb9@\xa4\xcc\x1a\x10\xc9\xdb\x00"U\'\x81H}k\xb0x(\x88T\xd7\x07\x92\x8c!\xf7@l\x85"\x10\xa94\x05\xcc\xfe\x00V\xc3\xc3\xc0\xc4\xf8?\xc8\x03l2\x03\xe3(I\x0c\t\x00\x89\xd1\x0b\xd5\nendstream\nendobj\n                                                                                                                                                                   